# Import and Setup

In [1]:
import datetime as dt
import pandas as pd
import numpy as np
import sys

import glob
import matplotlib.colors as colors
import matplotlib.pyplot as plt 
from mpl_toolkits.axes_grid1 import make_axes_locatable
params = {'font.size':14,'font.family':'lmodern','xtick.labelsize':16, 'ytick.labelsize':16,}

# plt.style.use('../../style.mplstyle')
# colors = plt.rcParams['axes.prop_cycle'].by_key()['color']

sys.path.insert(0, '../../../purity-monitor/')
sys.path.insert(0, '../../WaveformAnalysis/')
import Dataset as Dataset
import StandMonitor as Monitor
import PlotFunctions as Plt
import importlib
importlib.reload(Dataset)
importlib.reload(Monitor)
importlib.reload(Plt)
import scipy as sp
from scipy.optimize import curve_fit
from scipy.stats import norm
from scipy.special import erfc
import scipy.signal as spsig

# Data Read-In

In [2]:
D1 = Dataset.Dataset(Path='/project/david_moore/aj487/Data_WL110/LXe_Setup/SiPM/20210914/', Selection='*.h5')
files1 = glob.glob('/project/david_moore/aj487/Data_WL110/LXe_Setup/SiPM/20210913/*')
#files2 = glob.glob('/project/david_moore/aj487/Data_WL110/LXe_Setup/SiPM/20210914/*')
#for file in files2:
#     files1.append(file)

#D1.Files = files1
#D1.Files = files2
for File in D1.Files:
    print(File)
    D1.ImportDataFromHDF5(File, D1.Ch)

/project/david_moore/aj487/Data_WL110/LXe_Setup/SiPM/20210914/SiPM_33V_-99C_LED_trigger_8.h5


AttributeError: 'str' object has no attribute 'decode'

In [ ]:
Print=True
# fig = plt.figure(figsize=(12,8))
time=np.linspace(-5,5,1256)
for ii, ch in enumerate(D1.Ch):
#     print(" | Processing data in channel %d..." % (ch.ID))
#     if ii==1:
    ch.GetSampling()
#     print(ch.Amp)
#     ch.Amp = [x for _, x in sorted(zip(ch.TimeStamp, ch.Amp))]
    ch.Amp = np.array(ch.Amp)
#     print (ch.Amp)
    ch.TimeStamp = np.array(sorted(ch.TimeStamp))

    ch.Amp = ch.SubtractBaseline(Data=ch.Amp, state=Print)
#         plt.plot(time,ch.Amp[5],label='Before Filter')  

    #ch.Amp = ch.RemoveNoise(Data=ch.Amp, HighPass=2000000, state=Print)
#         plt.plot(time,ch.Amp[5],label='After Filter') 
#         plt.legend()
#         plt.show()
    ch.GetAllMaxima(Data=ch.Amp, state=Print)# Look at the code for the time window. Add a time window argument
    ch.GetIntegral(Data=ch.Amp, state=Print)# Look at the code for the time window. Add a time window argument
#         ch.RunFit(Data=D1.Ch[1].Amp)

In [ ]:
print("Time=",D1.Ch[0].Time)
print("Integral=",D1.Ch[0].Integral)
print("Samples=",D1.Ch[0].Samples)
print('Total Time=',D1.Ch[0].TotalTime,'us')
print("Sampling=",D1.Ch[0].Sampling)
print("Size Amplitude=",len(D1.Ch[0].Amp[2]))

Sampling_Frequency=1000000/((D1.Ch[0].Time[1])-(D1.Ch[0].Time[0]))
Time_Min=D1.Ch[0].Time.min()
Time_Max=D1.Ch[0].Time.max()
print("Time Min ($/mu$s)=",Time_Min)
print("Time Max ($/mu$s)=",Time_Max)
print("Sampling Frequency (per second)=",Sampling_Frequency)

In [ ]:
print(D1.Ch[0].Amp[0])

In [ ]:
def func(x,Base,V0,sigma,tau,mu):
        return Base + 0.5*V0 * np.exp(0.5 * (sigma/tau)**2 - (x-mu)/tau) * erfc(1/np.sqrt(2)* (sigma/tau - (x-mu)/sigma))
fig = plt.figure(figsize=(12,8))

plt.plot(D1.Ch[1].Time,-D1.Ch[1].Amp[1],color='r',lw=1)

xdata=D1.Ch[1].Time
ydata=-D1.Ch[1].Amp[1]
        
popt, pcov = curve_fit(func, xdata, ydata, maxfev=100000000,p0=[0,100,0.5,0.5,0.5])
        
# print("integral_curve=",D1.Ch[1].GetIntegral(Data=func(xdata,*popt), state=Print))
plt.plot(xdata,func(xdata,*popt),'--',lw=3)
print ("Base=",popt[0],"V0=",popt[1],"sigma=",popt[2],"tau=",popt[3],"mu=",popt[4])



plt.xlabel('Time ($\mu$s)')
plt.ylabel('Amplitude (mV)')
# plt.xlim(-2,2)
# plt.ylim(0,500)
# plt.legend(loc=2,fontsize=14,framealpha=0.8)
plt.grid()
plt.show()


In [ ]:
fig = plt.figure(figsize=(12,8))
filtfig = plt.figure(figsize=(12,8))

time=np.arange(Time_Min,Time_Max,1000000/Sampling_Frequency)
maxima= []
itime = 0.55
integ = []
int_wind = [0.4,0.8]
scale_fac = 38
fitamp = []
pescale = 187.2 # from fit

def func(x,V0):
        sigma,tau,mu=0.1,0.23,0.4
        return V0 * np.exp(0.5 * (sigma/tau)**2 - (x-mu)/tau) * erfc(1/np.sqrt(2)* (sigma/tau - (x-mu)/sigma))

flow = 2*3e5/Sampling_Frequency
fhigh = 2*2e6/Sampling_Frequency
b,a = spsig.butter(3,[flow,fhigh],btype="bandpass")

for ii,amp in enumerate(D1.Ch[1].Amp):
    amp = amp*-1

    if len(amp)==2512:
        pass
    else:
        plt.figure(fig.number)
        plt.plot(time,amp, color='k', lw=1, alpha=0.1)
        plt.figure(filtfig.number)
        plt.plot(time,filtamp, color='k', lw=1, alpha=0.1)
        #maxima.append(np.max(amp))
        
        #peak value (filtered)
        filtamp = spsig.filtfilt(b,a,amp)
        maxima.append(np.interp(itime, time, filtamp)/pescale)
        
        #integral
        gpts = np.logical_and(time>int_wind[0], time<int_wind[1])
        integ.append(np.sum(amp[gpts])/scale_fac)
        
        #fit
        popt, pcov = curve_fit(func, time, amp ,p0=[1250,])
        fitamp.append(popt[0])
        
        if(False and ii % 100 == 0):
            plt.figure()
            plt.plot(time, amp)
            plt.plot(time, func(time,*popt))
            plt.show()
        

plt.figure(fig.number)
plt.xlim(0,1.5)
plt.ylim(-500,1500)
plt.title("Voltage Waveform for FBK HD3 SiPM")
plt.xlabel('Time ($\mu$s)')
plt.ylabel('Amplitude (mV)')
plt.grid()

plt.figure(filtfig.number)
plt.xlim(0,1.5)
plt.ylim(-500,1500)
plt.title("Voltage Waveform for FBK HD3 SiPM, filtered")
plt.xlabel('Time ($\mu$s)')
plt.ylabel('Amplitude (mV)')
plt.grid()

## now make amplitude plot
h_amp,b_amp = np.histogram(maxima, bins=70, range=(-1,6))
h_int,b_int = np.histogram(integ, bins=70, range=(-250,2000))
h_fit,b_fit = np.histogram(fitamp, bins=70, range=(-250,2000))

bc_amp = b_amp[:-1] + np.diff(b_amp)/2

def gfun(x,A,mu,sig):
    return A*np.exp(-(x-mu)**2/(2*sig**2))

gpts = np.logical_and(bc_amp>0.5,bc_amp<1.4)
po,pc = curve_fit(gfun, bc_amp[gpts], h_amp[gpts], p0=[100,1,0.3])
print(po[2]/po[1])

xx=np.linspace(0.6,1.4,100)

plt.figure()
plt.step(b_amp[:-1], h_amp, where='post')
plt.plot(xx, gfun(xx,*po),'r', label="$\sigma$ = %.2f PE"%po[2])
#plt.step(b_int[:-1], h_int, where='post')
#plt.step(b_fit[:-1], h_fit, where='post')
plt.xlabel("PE")
plt.ylabel("Counts")
plt.legend()

plt.figure()
plt.step(b_amp[:-1], h_amp, where='post')
plt.plot(xx, gfun(xx,*po),'r', label="$\sigma$ = %.2f PE"%po[2])
#plt.step(b_int[:-1], h_int, where='post')
#plt.step(b_fit[:-1], h_fit, where='post')
plt.gca().set_yscale('log')
plt.xlabel("PE")
plt.ylabel("Counts")

plt.show()
# fig.savefig('Waveform_highpass_2000000.pdf', dpi=500)

In [ ]:
xdata=[]
ydata=[]
popt=[]
pcov=[]


V0=[]
sigma=[]
tau=[]
mu=[]

func_data=[]

def func(x,V0):
        sigma,tau,mu=0.13,0.18,0.6
        return 0.5*V0 * np.exp(0.5 * (sigma/tau)**2 - (x-mu)/tau) * erfc(1/np.sqrt(2)* (sigma/tau - (x-mu)/sigma))

# fig = plt.figure(figsize=(12,8))

for ii,amp in enumerate(D1.Ch[1].Amp):
    amp=-1*amp
    if len(amp)==2512:
        pass
    else:
        
        xdata=D1.Ch[0].Time
        ydata=amp
#         plt.figure()
#         plt.plot(xdata,ydata)
#         plt.plot(xdata,func(xdata,1250))
#         plt.show()
        popt, pcov = curve_fit(func, xdata, ydata,p0=[1250])
        
        
        
        V0.append(popt[0])
#         sigma.append(popt[1])
#         tau.append(popt[2])
#         mu.append(popt[3])
        func_data.append(func(xdata,*popt))
#         plt.plot(xdata,func(xdata,*popt),label=popt[0])
#         plt.legend()
#         plt.show()
ch.GetIntegral(Data=func_data,state=Print)
# plt.xlabel('Time ($\mu$s)')
# plt.ylabel('Amplitude (mV)')
# plt.xlim(-2,2)
# # plt.ylim(0,500)
# plt.legend(loc=2,fontsize=14,framealpha=0.8)
# plt.grid()
# plt.show()

# print(Amplitude)
# print (mu)
# print (sigma)
# print(A)

In [ ]:
fig = plt.figure(figsize=(12,8))

BINS=np.linspace(0,70000,70)
plt.hist(D1.Ch[1].Integral, bins=BINS, histtype='step',lw=2)

plt.title("Histogram of Integrals between 0.5$\mu$s and 1.0$\mu$s from fit function")
plt.xlabel('Integral')
plt.ylabel('Number of entries')
# plt.yscale('log')

plt.grid()
plt.show()


In [ ]:
Amplitude=((np.array(V0)/0.5))
fig = plt.figure(figsize=(12,8))

BINS=np.linspace(0,4000,20)
plt.hist(Amplitude, bins=BINS, histtype='step',lw=2,label='Amplitude')

# plt.yscale('log')
plt.grid()
plt.title("Histogram of Peak Amplitudes from fit function")
plt.xlabel('Peak Amplitude (mV)')
plt.ylabel('Number of Entries')
plt.legend(loc=1,fontsize=14,framealpha=0.8)
plt.show()


In [ ]:
fig = plt.figure(figsize=(12,8))

normalized_wf=[]
for ii,amp in enumerate(D1.Ch[1].Amp):
    amp = amp*-1
    if len(amp)==2512:
        pass
    else:
        normalized_wf.append(amp/len(amp))
        plt.plot(time,amp/len(amp), color='k', lw=0.1, alpha=0.1)        
# plt.xlim(0,2)
plt.ylim(0,2)
plt.title("Normalized Voltage Waveform for FBK HD3 SiPM")
plt.xlabel('Time ($\mu$s)')
plt.ylabel('Amplitude (mV)')
plt.grid()
plt.show()
fig.savefig('Waveform_highpass_2000000_normalized.pdf', dpi=500)
normalized_wf=np.asarray(normalized_wf)
print(normalized_wf.shape)
# print (normalized_wf)

In [ ]:
fig = plt.figure(figsize=(12,8))

dt=1/D1.Ch[0].Sampling
n = len(D1.Ch[0].Amp[2])
fhat = np.fft.fft(normalized_wf, n) #computes the fft
psd = fhat * np.conj(fhat)/n
freq = (1/(dt*n)) * np.arange(n) #frequency array
idxs_half = np.arange(1, np.floor(n/2), dtype=np.int32) #first half index

plt.plot(freq[idxs_half],np.abs(psd[idxs_half]),color='k',lw=0.1)
plt.title("FFT of the SiPM Waveform")
plt.xlabel('Frequency (Hz)')
plt.ylabel('Amplitude')
# plt.grid()
plt.show()
fig.savefig('Waveform_FFT_new_method.pdf', dpi=500)

In [ ]:
psd_idxs=(psd<20)
psd_clean = psd * psd_idxs #zero out all the unnecessary powers
fhat_clean = psd_idxs * fhat #used to retrieve the signal
signal_filtered = np.fft.ifft(fhat_clean) #inverse fourier transform

In [ ]:
fig = plt.figure(figsize=(12,8))
plt.plot(freq[idxs_half],np.abs(psd_clean[idxs_half]),color='k',lw=0.1)
plt.title("Filtered FFT of the SiPM Waveform")
plt.xlabel('Frequency (Hz)')
plt.ylabel('Amplitude')
plt.grid()
plt.show()
# fig.savefig('Waveform_FFT_filtered_new_method.pdf', dpi=500)

In [ ]:
fig = plt.figure(figsize=(12,8))

time=np.linspace(Time_Min,Time_Max,2512)
for waveforms in signal_filtered:
    plt.plot(time,np.abs(waveforms), color='k', lw=0.1, alpha=0.1)
    
plt.title("Filtered SiPM Waveform 0-10")
plt.xlabel('Time ($\mu$s)')
plt.ylabel('Amplitude (mV)')
# plt.xlim(0,2)
# plt.ylim(0,2)
plt.grid()
plt.show()
# fig.savefig('Waveform_highpass_2000000_filtered0-10.pdf', dpi=500)

In [ ]:
xdata=[]
ydata=[]
popt=[]
pcov=[]

Base=[]
V0=[]
sigma=[]
tau=[]
mu=[]

func_data=[]

def func(x,V0):
        sigma,tau,mu=0.13,0.18,0.6
        return 0.5*V0 * np.exp(0.5 * (sigma/tau)**2 - (x-mu)/tau) * erfc(1/np.sqrt(2)* (sigma/tau - (x-mu)/sigma))


for waveforms in signal_filtered:
    xdata=time
    ydata=np.abs(waveforms)
    popt, pcov = curve_fit(func, xdata, ydata,p0=[1250])
    V0.append(popt[0])
    func_data.append(func(xdata,*popt))

ch.GetIntegral(Data=func_data,state=Print)

In [ ]:
fig = plt.figure(figsize=(12,8))

BINS=np.linspace(0,50,250)
plt.hist(D1.Ch[1].Integral, bins=BINS, histtype='step',lw=2)
plt.title("Histogram of Integrals between 0.5$\mu$s and 1.0$\mu$s from fit function")
plt.xlabel('Integral')
plt.ylabel('Number of entries')
plt.yscale('log')
plt.grid()
plt.show()

In [ ]:
Amplitude=((np.array(V0)/0.5))
fig = plt.figure(figsize=(12,8))

BINS=np.linspace(0,4000,20)
plt.hist(Amplitude, bins=BINS, histtype='step',lw=2,label='Amplitude')

# plt.yscale('log')
plt.grid()
plt.title("Histogram of Peak Amplitudes from fit function")
plt.xlabel('Peak Amplitude (mV)')
plt.ylabel('Number of Entries')
plt.legend(loc=1,fontsize=14,framealpha=0.8)
plt.show()
